# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Choose global variables:

In [1]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, theta, gamma, p, seed):
    return [n, alpha, beta, theta, gamma, p, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), theta=(0.0, 0.5), gamma=(0.0, 2.0), p=(0.0, 1.0), seed=(1, 999))
display(params)

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), FloatSlider(value=1.0, description=…

### Set global variables:
(Note: will break if you set alpha = 0 due to 0 division error)
Remember to rerun this cell every time a slider is updated

In [2]:
n, alpha, beta, theta, gamma, p, seed = params.result

### Define particle and environment classes:

In [21]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


class Particle:
    
    count = 0
    
    def __init__(self, time, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        self._time = time # time at which the particle went to this position
        
    def update_position(self, time, value=1):
        self._position += value
        self._time = time # update time
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        Particle.count = 0
        curr_time = 0
        actions = []
        entrances = 0
        exits = 0
        travel_times = []
        cell_speeds = []
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle(curr_time, gamma=gamma, p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                if self.occupied_positions[self.n-1]:
                    end_particle = self.particles.pop(0)
                    self.occupied_positions[self.n-1] = 0
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == self.n - 1 or self.occupied_positions[particle_pos + 1] == 1: # changed to refer to self.n instead of n
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    
                    cell_speeds.append(1 / (curr_time - particle._time))
                    particle.update_position(curr_time)
                    actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {}'.format(sum(cell_speeds) / len(cell_speeds)))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [48]:
class Particle2D(Particle):
    def __init__(self, gamma=gamma, p=p, direction=0): # 0 is horizontal, 1 is vertical
        Particle.__init__(self, gamma, p)
        self._position = [-1, -1]
        self._direction = direction
        
    def update_position(self, time, value=1):
        self._time = time
        self._position[self._direction] += value
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_direction(self):
        return self._direction
        
# FIRST INDEX COLUMN, SECOND INDEX ROAD
class Env2D(Environment):
    def __init__(self, num_roads=(1,1), n=n, alpha=alpha, beta=beta, theta=theta):
        Environment.__init__(self, n, alpha, beta)
        self.theta = theta
        self.occupied_positions = np.array([[0 for _ in range(n)] for _ in range(n)])
        self.horizontal_roads = np.random.choice(range(n), num_roads[0], replace=False)
        self.vertical_roads = np.random.choice(range(n), num_roads[1], replace=False)
        
        self.num_roads = num_roads
        self.intersections = []
        [[self.intersections.append((x, y)) for x in self.vertical_roads] for y in self.horizontal_roads]
        self.inter_directions = {}
        for i in self.intersections:
            self.inter_directions[i] = 0
        
    def run_simulation(self, time_len=60):
        Particle.count = 0
        
#         print(self.occupied_positions)
        
        curr_time = 0
#         time_lapse = np.random.exponential(scale=1/(self.alpha * sum(self.num_roads)))
#         time.sleep(time_lapse)
#         curr_time += time_lapse
#         particle = Particle2D(gamma=gamma, p=p, direction=np.random.choice([0, 1], p=[self.num_roads[0]/sum(self.num_roads), self.num_roads[1]/sum(self.num_roads)]))
#         self.particles.append(particle)
#         if not particle.get_direction():
#             road = np.random.choice(self.horizontal_roads)
#             particle.set_position([0, road])
#             self.occupied_positions[0, road] = 1
#         else:
#             road = np.random.choice(self.vertical_roads)
#             particle.set_position([road, 0])
#             self.occupied_positions[road, 0] = 1
        
        actions = []#[(curr_time, particle.id, particle.get_position())]
        entrances = 0#1
        exits = 0
        travel_times = []
        cell_speeds = []
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + (self.alpha + self.beta) * sum(self.num_roads) + self.theta * (sum(self.num_roads) ** 2)  
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha for _ in range(sum(self.num_roads))] + [self.beta for _ in range(sum(self.num_roads))] + [self.theta for _ in range(sum(self.num_roads) ** 2)]) / merged_param
            particle = np.random.choice(self.particles + ['entrance' for _ in range(sum(self.num_roads))] + ['exit' for _ in range(sum(self.num_roads))] + ['intersection' for _ in range(sum(self.num_roads) ** 2)], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[0][road]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[0][road] = 1
                    new_particle.set_position([0, road])
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][0]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[road][0] = 1
                    new_particle.set_position([road, 0])
                
                travel_times.append([curr_time, None])
                actions += [(curr_time, new_particle.id, new_particle.get_position())]
                entrances += 1
                    
            elif particle == 'exit':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[self.n-1][road]:
                        particle = [p for p in self.particles if p.get_position() == [self.n-1, road]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[self.n-1][road] = 0
                        actions += [(curr_time, particle.id, [n, road])]
                        exits += 1
                        travel_times[particle.id][1] = curr_time
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][self.n-1]:
                        particle = [p for p in self.particles if p.get_position() == [road, self.n-1]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[road][self.n-1] = 0
                        actions += [(curr_time, particle.id, [road, n])]
                        exits += 1
                        travel_times[particle.id][1] = curr_time
            elif particle == 'intersection':
                i = np.random.choice(range(len(self.intersections)))
                intersection = self.intersections[i]
                self.inter_directions[intersection] = 0 if self.inter_directions[intersection] else 1
                actions += [(curr_time, 'intersection' + str(intersection), self.inter_directions[intersection])] 
            else:
                particle_pos = particle.get_position()
                direction = particle.get_direction()
                if not direction:
                    pos = (particle_pos[0] + 1, particle_pos[1])
                    if pos in self.intersections and self.inter_directions[pos]:
                        continue
                    elif particle_pos[0] == self.n - 1 or self.occupied_positions[particle_pos[0] + 1, particle_pos[1]] == 1:
                        continue
                    else:
                        cell_speeds.append(1 / (curr_time - particle._time))
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0] + 1][particle_pos[1]] = 1
                        particle.update_position(curr_time)
                        actions += [(curr_time, particle.id, particle.get_position())]
                else:
                    pos = (particle_pos[0], particle_pos[1] + 1)
                    if pos in self.intersections and not self.inter_directions[pos]:
                        continue
                    elif particle_pos[1] == self.n - 1 or self.occupied_positions[particle_pos[0], particle_pos[1] + 1] == 1:
                        continue
                    else:
                        cell_speeds.append(1 / (curr_time - particle._time))
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0]][particle_pos[1] + 1] = 1
                        particle.update_position(curr_time)
                        actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
#             print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {} seconds'.format(round(sum(travel_times) / len(travel_times), 3)))
        else:
            print('No cars reached the end')
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Multiple Lanes Environment:

In [15]:
class ParticleManyLane(Particle):
    def __init__(self, time, lane, gamma=gamma, p=p):
        Particle.__init__(self, time, gamma, p)
        self._position = [lane, 0] #first index lane, second index position
        
    def update_position(self, time, value=(0, 1)):
        self._time = time
        self._position[0] += value[0]
        self._position[1] += value[1]
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_position(self):
        return self._position[0], self._position[1]
    
class EnvManyLane(Environment):
    def __init__(self, n=n, alpha=alpha, beta=beta, lanes=1, delta=0.5):
        Environment.__init__(self, n, alpha, beta)
        self.lanes = lanes
        self.occupied_positions = np.zeros((lanes, n))
        self.delta = delta # tendency to change lanes
        #first index lane, second index position
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 
        curr_time = 0
        entrances = 0
        exits = 0
        actions = []
        travel_times = [] #[[0, None]]
        cell_speeds = []
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.lanes * self.alpha + self.lanes * self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] +  [self.lanes * self.alpha, self.lanes * self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                lane = np.random.randint(0, self.lanes)
                if self.occupied_positions[lane, 0]:
                    continue
                else:
                    new_particle = ParticleManyLane(curr_time, lane, gamma=gamma, p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[lane, 0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                lane = np.random.randint(0, self.lanes)
                if self.occupied_positions[lane, self.n-1]: 
                    for i in range(len(self.particles)):
                        particle = self.particles[i]
                        if particle.get_position() == (lane, self.n-1):
                            end_particle = particle
                            self.particles.pop(i)
                            break
                    self.occupied_positions[lane, self.n-1] = 0 
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_lane, particle_pos = particle.get_position()
                options = []
                if particle_pos != self.n - 1 and self.occupied_positions[particle_lane, particle_pos + 1] != 1:
                    options.append((0, 1)) # advance
                if particle_lane - 1 >= 0 and self.occupied_positions[particle_lane - 1, particle_pos - 1] == 0 and self.occupied_positions[particle_lane - 1, particle_pos] == 0:
                    options.append((-1, 0)) # left
                if particle_lane + 1 < self.lanes and self.occupied_positions[particle_lane + 1, particle_pos - 1] == 0 and self.occupied_positions[particle_lane + 1, particle_pos] == 0:
                    options.append((1, 0)) # right
                    
                if len(options) == 0:
                    continue
                else:
                    probabilities = []
                    # see which options are available and pick with probability proportional to the number of open spaces
                    # proportional to number of cars in front
                    if (0, 1) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane, particle_pos:])) * (1 - self.delta))
                    if (-1, 0) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane - 1, particle_pos:])) * self.delta)
                    if (1, 0) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane + 1, particle_pos:])) * self.delta)
                    probabilities = probabilities / sum(probabilities)
                    # choose option and update model
                    option = options[np.random.choice(len(options), p=probabilities)]
                    self.occupied_positions[particle_lane, particle_pos] = 0
                    self.occupied_positions[particle_lane + option[0], particle_pos + option[1]] = 1
                    if option == (0, 1):
                        cell_speeds.append(1 / (curr_time - particle._time))
                        particle.update_position(curr_time, (0, 1))
                    else:
                        particle.update_position(particle._time, (option))
                    actions += [(curr_time, particle.id, particle.get_position())]
                          
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
#             print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {} seconds'.format(round(sum(travel_times) / len(travel_times), 3)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {} cells per second'.format(round(sum(cell_speeds) / len(cell_speeds), 3)))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(round(entry[0], 2)), end = " ")
            print('Particle: {}'.format(entry[1]), end = " ")
            print('New lane, position: ({},{})'.format(entry[2][0], entry[2][1]), end = " ")
            print('\n')

### Run simulator:

In [22]:
env1 = Environment(n=n, alpha=alpha, beta=beta)
env1.run_simulation(time_len = 60)

[1, 1, 1, 0, 0, 1, 0, 0, 0, 0]
 done
Summary:
Total entrances: 11
Total exits: 7
Travel times: [17.13, 29.68, 28.38, 24.61, 28.69, 34.95, 27.95]
Average travel time: 27.340513885979934
Average cell speed: 1.3153632297521398

 Log:
Time: 1.2516195947367859
Particle: 0
New position: 0


Time: 5.962662040160619
Particle: 0
New position: 1


Time: 6.798072618285856
Particle: 1
New position: 0


Time: 6.803388270417819
Particle: 0
New position: 2


Time: 7.818740212782151
Particle: 0
New position: 3


Time: 8.5617788384401
Particle: 0
New position: 4


Time: 10.001336987505617
Particle: 1
New position: 1


Time: 10.602085954943533
Particle: 2
New position: 0


Time: 11.525627168535065
Particle: 1
New position: 2


Time: 12.921019511648852
Particle: 0
New position: 5


Time: 13.919854987784381
Particle: 0
New position: 6


Time: 14.340467460132503
Particle: 0
New position: 7


Time: 14.706481978256235
Particle: 0
New position: 8


Time: 15.771738229591843
Particle: 1
New position: 3


Time: 

In [49]:
env = Env2D(num_roads=(1, 1), n=n, alpha=alpha, beta=beta, theta=theta)
env.run_simulation(time_len = 60)

[[0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

 done
Summary:
Total entrances: 12
Total exits: 2
Average travel time: 46.84 seconds

 Log:
Time: 0.7004401755934768
Particle: intersection(7, 5)
New position: 1


Time: 0.7387129934680375
Particle: 0
New position: [7, 5]


Time: 0.884230379284212
Particle: 1
New position: [7, 9]


Time: 1.6316372190013813
Particle: intersection(7, 5)
New position: 0


Time: 1.7468336215372027
Particle: intersection(7, 5)
New position: 1


Time: 1.7613745870913171
Particle: 1
New position: [7, 9]


Time: 2.401140278054307
Particle: 2
New position: [7, 9]


Time: 4.632818009863344
Particle: 1
New position: [7, 9]


Time: 4.996621824964947
Particle: 2
New position: [7, 9]


Time: 5.060090278863482
Particle: intersection(7, 5)
New position: 0


Time: 5.240958209731411
Particle:

In [16]:
env2 = EnvManyLane(n=n, alpha=alpha, beta=beta, lanes=2)
env2.run_simulation(time_len = 60)

[[1. 1. 1. 1. 0. 1. 1. 0. 1. 0.]
 [1. 1. 0. 0. 1. 0. 0. 1. 0. 0.]]

 done
Summary:
Total entrances: 24
Total exits: 13
Average travel time: 24.778 seconds
Average cell speed: 1.046 cells per second

 Log:
Time: 0.14 Particle: 0 New lane, position: (1,0) 

Time: 1.04 Particle: 1 New lane, position: (0,0) 

Time: 1.82 Particle: 0 New lane, position: (1,1) 

Time: 2.02 Particle: 2 New lane, position: (1,0) 

Time: 3.1 Particle: 1 New lane, position: (0,1) 

Time: 3.24 Particle: 0 New lane, position: (1,2) 

Time: 3.64 Particle: 1 New lane, position: (0,2) 

Time: 3.88 Particle: 3 New lane, position: (0,0) 

Time: 4.27 Particle: 3 New lane, position: (0,1) 

Time: 4.31 Particle: 1 New lane, position: (0,3) 

Time: 4.33 Particle: 2 New lane, position: (0,0) 

Time: 4.39 Particle: 3 New lane, position: (0,2) 

Time: 4.46 Particle: 4 New lane, position: (1,0) 

Time: 4.61 Particle: 2 New lane, position: (0,1) 

Time: 4.69 Particle: 5 New lane, position: (0,0) 

Time: 5.52 Particle: 4 New lane